In [1]:
from context import *
from stable_baselines3 import PPO,A2C,SAC,TD3,DQN,DDPG
from stable_baselines3.common.save_util import load_from_zip_file
from stable_baselines3.common.monitor import Monitor
import torch as th
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

from pprint import pprint

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-04-02 13:13:32.113182: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-02 13:13:32.122284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-02 13:13:32.132199: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Atte

In [2]:
import enum

class EnvMode(enum.Enum):
    VANILLA=0
    STL=1

In [3]:
import rlrom.wrappers.stl_wrapper
import stlrom
from rlrom.envs import *

def make_env(train=True, env_mode=EnvMode.VANILLA):
    if train:
        env = gym.make("highway-fast-v0")
    else:
        env = gym.make("highway-v0", render_mode='human')

    env.unwrapped.configure({
            "observation": {"type": "Kinematics"},
                "action": {
                    "type": "DiscreteMetaAction",
                },
                "lanes_count": 4,
                "vehicles_count": 50,
                "controlled_vehicles": 1,
                "initial_lane_id": None,
                "duration": 100,  # [s]
                "ego_spacing": 2,
                "vehicles_density": 1,
                "collision_reward": -.1,  # The reward received when colliding with a vehicle.
                "right_lane_reward": 0,  # The reward received when driving on the right-most lanes, linearly mapped to
                # zero for other lanes.
                "high_speed_reward": 1.,  # The reward received when driving at full speed, linearly mapped to zero for
                # lower speeds according to config["reward_speed_range"].
                "lane_change_reward": 0,  # The reward received at each lane change action.
                "reward_speed_range": [20, 30],
                "normalize_reward": True,
                "offroad_terminal": False,        
    })

    if env_mode==EnvMode.STL:
        cfg = cfg_envs['highway-env']
        driver= stlrom.STLDriver()
        driver.parse_string(cfg['specs'])
        env = rlrom.wrappers.stl_wrapper.STLWrapper(env,driver,signals_map=cfg)

    return env

# Training vanilla highway-env

In [50]:
neurons = 128

policy_kwargs = dict(
    #activation_fn=th.nn.ReLU,
    net_arch=dict(pi=[neurons, neurons], qf=[neurons, neurons])
)



In [51]:
n_cpu = 12
batch_size = 64
#env = make_env()
vec_env = make_vec_env(make_env, n_envs=n_cpu, vec_env_cls=SubprocVecEnv)
model = PPO(
     "MlpPolicy",
     vec_env,
     policy_kwargs=policy_kwargs,
     n_steps=batch_size * 12 // n_cpu,
     batch_size=batch_size,
     n_epochs=10,
     learning_rate=5e-4,
     gamma=0.99,
     verbose=1,
     tensorboard_log="./highway_ppo/"
)



/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is n

Using cuda device


In [ ]:
# Train the agent
model.learn(
    total_timesteps=50_000,
    progress_bar=True
)


In [61]:
model.save('ppo_model')

# Testing

In [4]:
model = PPO.load('ppo_model.zip')

In [15]:
env = make_env(train=False,env_mode=EnvMode.STL)
obs, info = env.reset()
wobs = env.wrapped_obs

In [16]:
action, _states = model.predict(wobs, deterministic=True)
    

In [17]:
obs, reward, terminated, truncated, info = env.step(action)    


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [11]:
for _ in range(200):    
    action, _states = model.predict(wobs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)    
    wobs= env.wrapped_obs
    if terminated or truncated:
        obs, info = env.reset()
env.close()

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [38]:
env.close()

In [ ]:
model = PPO.load('ppo_model.zip')
T.model = model
T.test_seed(0, 1000)

Testing seed  0  with model  RandomAgent  for  1000  steps  lazy:  True


9.620146609914984

In [36]:
print(obs)
obs[0][0]

[[ 1.0000000e+00  1.0000000e+00  2.2204460e-16  2.5000000e-01
   0.0000000e+00]
 [ 1.0000000e+00  2.3241380e-02  2.5000000e-01 -3.1315042e-03
   0.0000000e+00]
 [ 1.0000000e+00  3.6703035e-02 -1.1102230e-16 -5.9754923e-03
   0.0000000e+00]
 [ 1.0000000e+00  9.4727471e-02  7.5000000e-01  1.5306712e-02
   0.0000000e+00]
 [ 1.0000000e+00  1.3697396e-01  5.0000000e-01  1.6223991e-02
   0.0000000e+00]]


1.0

In [ ]:
from bokeh.plotting import figure,show
from bokeh.io import output_notebook
output_notebook()

In [ ]:
lay = """
action
ego_x, car1_x
"""
fig, status = T.get_fig(lay)
show(fig)
